In [1]:
import requests
import polygon

client = polygon.RESTClient("NbOaYW2hvhGCEKNKaUeVI3iNg1P40Sie")

In [2]:
us_ticker_list_fpath = '/student/wangsaizhuo/Data/my_data_dir/main/us/instruments/all.txt'
with open(us_ticker_list_fpath) as f:
    us_ticker_list = f.read().splitlines()
us_stocks = [x.split('\t')[0] for x in us_ticker_list]

In [5]:
# For a frequency, get aggregate bars for all stocks
import pandas as pd
from datetime import datetime


def get_aggs_for_all_stocks(freq, start, end):
    print(f"Getting {freq} bars for all stocks from {start} to {end}")
    all_aggs = {}

    def stock_worker(stock) -> pd.Series:
        client = polygon.RESTClient("NbOaYW2hvhGCEKNKaUeVI3iNg1P40Sie")
        aggs = client.get_aggs(
            ticker=stock, multiplier=1, timespan=freq, from_=start, to=end
        )
        # for each field, return a series
        time_indices = [
            datetime.fromtimestamp(x.timestamp / 1000.0) for x in aggs
        ]
        time_indices = [x.strftime("%Y%m%dT%H%M%S") for x in time_indices]
        open_prices = [x.open for x in aggs]
        high_prices = [x.high for x in aggs]
        low_prices = [x.low for x in aggs]
        close_prices = [x.close for x in aggs]
        volumes = [x.volume for x in aggs]
        vwaps = [x.vwap for x in aggs]
        # amounts = [x.volume * x.vwap for x in aggs]
        ntrades = [x.transactions for x in aggs]
        series_dict = {
            "open": pd.Series(open_prices, index=time_indices, name=stock),
            "close": pd.Series(close_prices, index=time_indices, name=stock),
            "high": pd.Series(high_prices, index=time_indices, name=stock),
            "low": pd.Series(low_prices, index=time_indices, name=stock),
            "volume": pd.Series(volumes, index=time_indices, name=stock),
            "vwap": pd.Series(vwaps, index=time_indices, name=stock),
            # "amount": pd.Series(amounts, index=time_indices, name=stock),
            "ntrades": pd.Series(ntrades, index=time_indices, name=stock),
        }
        return series_dict

    import loky
    from tqdm import tqdm

    with loky.get_reusable_executor(128) as e:
        all_series = [x for x in tqdm(e.map(stock_worker, us_stocks), total_length=len(us_stocks))]
        # For each data field, make a dataframe with each column as a stock
        for field in all_series[0].keys():
            all_aggs[field] = pd.concat([x[field] for x in all_series], axis=1)
    return all_aggs

In [21]:
import os
day_vp_dir = '/student/wangsaizhuo/Data/my_data_dir/main/us/aggs'
files = os.listdir(day_vp_dir)
from tqdm import tqdm
data_dict = {}
for f in tqdm(files):
    if not f.endswith('.csv'):
        continue
    data = pd.read_csv(os.path.join(day_vp_dir, f), index_col=0)
    stock = f.split('.')[0]
    for col in data.columns:
        data_dict.setdefault(col, {})[stock] = data[col]

100%|██████████| 6375/6375 [00:25<00:00, 249.05it/s]


In [23]:
pd.DataFrame(data_dict['volume'])

,SIX,NIHDQ,SSNT,MOG-A,EPOW,JILL,FSNB,AEHL,RXRX,PLAO,...,MPTI,ABST,SHEN,VZ,ENZ,EBAY,WORX,RUSHB,TWLVU,SXT
20040102T130000,NaN,NaN,NaN,NaN,NaN,27774.8,NaN,NaN,NaN,NaN,...,NaN,NaN,48600.0,7281100.0,163590.0,4.327784e+07,NaN,35926.875,NaN,108800.0
20040105T130000,NaN,NaN,NaN,NaN,NaN,34897.0,NaN,NaN,NaN,NaN,...,NaN,NaN,61368.0,14689900.0,211890.0,6.230877e+07,NaN,25312.500,NaN,206000.0
20040106T130000,NaN,NaN,NaN,NaN,NaN,23967.4,NaN,NaN,NaN,NaN,...,NaN,NaN,25764.0,9881200.0,201495.0,4.030268e+07,NaN,68863.500,NaN,206200.0
20040107T130000,NaN,NaN,NaN,NaN,NaN,27367.4,NaN,NaN,NaN,NaN,...,NaN,NaN,171468.0,9364100.0,127785.0,3.068029e+07,NaN,82350.000,NaN,180800.0
20040108T130000,NaN,NaN,NaN,NaN,NaN,18720.4,NaN,NaN,NaN,NaN,...,NaN,NaN,76800.0,9237100.0,311220.0,3.843320e+07,NaN,79261.875,NaN,209900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240520T120000,541148.0,NaN,24528.0,NaN,4834.0,51616.0,NaN,71069.0,3484861.0,450768.0,...,32338.0,NaN,NaN,NaN,NaN,NaN,12149.0,NaN,NaN,NaN
20240521T120000,729688.0,NaN,22864.0,NaN,3146.0,22897.0,NaN,290539.0,3014821.0,201602.0,...,42222.0,NaN,NaN,NaN,NaN,NaN,19713.0,NaN,NaN,NaN
20240522T120000,691345.0,NaN,24738.0,NaN,3080.0,36742.0,NaN,69169.0,3587831.0,NaN,...,29156.0,NaN,NaN,NaN,NaN,NaN,32031.0,NaN,NaN,NaN
20240523T120000,943453.0,NaN,36614.0,NaN,6233.0,65920.0,NaN,21995.0,6667755.0,50613.0,...,40802.0,NaN,NaN,NaN,NaN,NaN,78568.0,NaN,NaN,NaN


In [10]:
df_dict = {}
os.makedirs('/student/wangsaizhuo/Data/my_data_dir/main/us/aggs/full', exist_ok=True)
for field in data_dict.keys():
    data = pd.DataFrame(data_dict[field])
    data.to_csv(f'/student/wangsaizhuo/Data/my_data_dir/main/us/aggs/full/{field}.csv')
    print(f"Saved {field}")
    df_dict[field] = data

Saved Unnamed: 0
Saved open
Saved close
Saved high
Saved low
Saved volume
Saved vwap
Saved ntrades


In [17]:
ticks = df_dict['Unnamed: 0']['AAPL']
ticks = ticks.values

In [19]:
df_dict['open']

,SIX,NIHDQ,SSNT,MOG-A,EPOW,JILL,FSNB,AEHL,RXRX,PLAO,...,MPTI,ABST,SHEN,VZ,ENZ,EBAY,WORX,RUSHB,TWLVU,SXT
0,2.1775,NaN,4.3000,NaN,2.230,63.65,9.80,23.701,30.0000,10.0300,...,13.30,11.00,4.4000,35.10,17.1429,13.5438,48.00,2.8652,10.0200,19.87
1,2.1675,NaN,4.2700,NaN,3.050,64.80,9.80,23.100,28.6774,10.0301,...,16.72,11.80,4.3167,35.50,17.2381,13.3228,60.45,2.8741,10.0662,20.06
2,2.2000,NaN,4.0000,NaN,2.590,66.80,9.80,24.200,27.3200,10.1000,...,14.04,11.73,4.3500,36.25,17.5238,13.5817,125.25,2.8385,10.0700,20.34
3,2.0575,NaN,4.2237,NaN,2.300,66.35,9.85,24.800,29.5000,10.0800,...,11.94,11.35,4.4084,36.45,17.6190,13.6953,100.95,2.9926,10.0600,20.08
4,2.0000,NaN,4.1000,NaN,2.181,66.90,9.80,24.500,33.6400,10.0200,...,12.00,11.60,4.4375,36.80,17.7619,13.7584,81.00,2.9393,10.0700,20.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23.8600,35.98,1.3200,40.6400,NaN,43.3600,NaN,58.32
4996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23.8500,35.70,1.3500,40.5100,NaN,43.5800,NaN,58.06
4997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23.2000,35.94,1.3300,37.2800,NaN,44.0000,NaN,58.02
4998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23.7000,35.87,1.2810,40.2900,NaN,44.5900,NaN,58.02
